# age|gender预测--TFIDF版

In [1]:
import numpy as np
import pandas as pd

In [2]:
import gc

## 载入用户点击日志

In [3]:
pd_user_ad_list = pd.read_csv("./data/pd_user_ad_id_seg_all_click_time.csv")

In [5]:
pd_user_ad_list.head()

,user_id,time,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,1,20 20 20 39 40 43 46 52 60 64 64 73 76,877468 209778 821396 1683713 122032 71691 1940...,1 1 1 1 1 1 1 1 1 1 1 1 2,773445 188507 724607 1458878 109959 66210 1678...,0 136 0 0 1334 0 0 0 0 64 1454 64 1261,5 2 5 5 2 18 5 5 18 2 2 2 2,29455 9702 7293 14668 11411 14681 17189 367 44...,106 6 326 326 0 326 73 217 64 245 238 245 6
1,2,10 11 14 17 28 28 28 38 38 39 41 42 42 42 44 4...,63441 155822 39714 609050 13069 1266180 441462...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,58788 139702 38066 541125 14495 1107111 392680...,87 80 129 129 1400 0 87 0 1261 0 111 129 0 0 0...,2 2 2 2 2 18 2 18 2 18 2 2 18 18 18 18 2 2 2 2...,22885 10686 18562 25932 768 34505 22885 26006 ...,318 238 6 6 317 47 318 47 6 47 242 6 47 6 47 4...
2,3,12 13 14 14 14 17 19 22 31 36 37 44 47 47 50 5...,661347 808612 593522 825434 710859 726940 3920...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,586668 713448 527601 728308 629278 643108 3502...,36256 40905 1674 35985 1674 0 0 1031 1786 2258...,17 17 2 17 2 18 18 2 2 2 18 12 2 2 18 18 8 4 2...,32974 9877 17018 14186 18492 9058 8371 2336 39...,0 0 322 0 322 6 54 6 322 322 205 302 322 322 2...
3,4,8 15 41 44 48 48 48 48 49 52 58 58 59 61 62 62...,39588 589886 574787 1892854 1230094 31070 2264...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,37966 524312 511235 1638619 1076286 30773 1953...,1862 0 2625 38743 39904 39422 37758 41265 0 15...,2 18 2 17 17 17 4 17 18 2 2 2 2 2 2 2 2 2 5 2 ...,19451 7976 13084 12130 10172 13299 811 23664 1...,238 25 248 0 0 0 0 0 88 319 6 238 319 238 319 ...
4,5,3 13 14 15 20 21 24 25 27 28 29 30 32 32 35 35...,296145 350759 24333 43235 852327 1054434 12964...,1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,265971 314795 24966 41148 751113 925792 113378...,0 0 87 136 0 136 0 0 0 1064 0 2620 0 0 0 136 2...,5 8 2 2 18 2 5 18 18 2 18 2 5 5 18 2 2 18 18 5...,11882 992 22885 9706 38760 2862 17745 31552 22...,297 0 318 6 322 6 288 322 319 238 322 322 203 ...


In [6]:
pd_user_ad_list.shape

(1900000, 9)

### 标签信息

In [7]:
df_user_label = pd.read_csv("./data/train_preliminary/user.csv")

In [8]:
df_user_label.shape

(900000, 3)

#### 分离出测试集用户id

In [9]:
user_ids_test = set(pd_user_ad_list['user_id'].unique())-set(df_user_label['user_id'].values)

In [10]:
len(user_ids_test)

1000000

In [11]:
df_test_user_id = pd.DataFrame(data=user_ids_test,columns=['user_id'])

In [12]:
df_test_user_id.shape

(1000000, 1)

In [13]:
df_test_user_id['user_id'].min(),df_test_user_id['user_id'].max()

(3000001, 4000000)

In [14]:
df_user_label['user_id'].min(),df_user_label['user_id'].max()

(1, 900000)

## 构造tfidf向量

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [18]:
#tf换成CountVectorizer即可
tf_idf_vectorizer_cr = TfidfVectorizer(min_df=30)
tf_idf_vectorizer_ad = TfidfVectorizer(min_df=30)
tf_idf_vectorizer_pr = TfidfVectorizer(min_df=30)
tf_idf_vectorizer_pc = TfidfVectorizer(min_df=30)
tf_idf_vectorizer_av = TfidfVectorizer(min_df=30)
tf_idf_vectorizer_in = TfidfVectorizer(min_df=30)

In [20]:
tf_idf_vectorizer_cr.fit(pd_user_ad_list['creative_id'].values)
tf_idf_vectorizer_ad.fit(pd_user_ad_list['ad_id'].values)
tf_idf_vectorizer_pr.fit(pd_user_ad_list['product_id'].values)
tf_idf_vectorizer_pc.fit(pd_user_ad_list['product_category'].values)
tf_idf_vectorizer_av.fit(pd_user_ad_list['advertiser_id'].values)
tf_idf_vectorizer_in.fit(pd_user_ad_list['industry'].values)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=30, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [21]:
len(tf_idf_vectorizer_cr.vocabulary_.keys()),len(tf_idf_vectorizer_ad.vocabulary_.keys()),len(tf_idf_vectorizer_pr.vocabulary_.keys()),len(tf_idf_vectorizer_pc.vocabulary_.keys()),len(tf_idf_vectorizer_av.vocabulary_.keys()),len(tf_idf_vectorizer_in.vocabulary_.keys())

(222544, 222625, 9934, 9, 23793, 286)

## 构造样本

In [22]:
pd_user_ad_list_train = pd.merge(pd_user_ad_list,df_user_label,on='user_id')

In [23]:
pd_user_ad_list_test = pd.merge(pd_user_ad_list,df_test_user_id,on='user_id')

In [24]:
pd_user_ad_list_train.shape,pd_user_ad_list_test.shape

((900000, 11), (1000000, 9))

In [25]:
gc.collect()

0

In [26]:
from scipy import sparse

In [27]:
tf_idf_train_cr = tf_idf_vectorizer_cr.transform(pd_user_ad_list_train['creative_id'].values)
tf_idf_train_ad = tf_idf_vectorizer_ad.transform(pd_user_ad_list_train['ad_id'].values)
tf_idf_train_pr = tf_idf_vectorizer_pr.transform(pd_user_ad_list_train['product_id'].values)
tf_idf_train_pc = tf_idf_vectorizer_pc.transform(pd_user_ad_list_train['product_category'].values)
tf_idf_train_av = tf_idf_vectorizer_av.transform(pd_user_ad_list_train['advertiser_id'].values)
tf_idf_train_in = tf_idf_vectorizer_in.transform(pd_user_ad_list_train['industry'].values)

In [28]:
tf_idf_train = sparse.hstack((tf_idf_train_cr,tf_idf_train_pr,tf_idf_train_pc,tf_idf_train_av,tf_idf_train_in)).tocsr()

In [29]:
tf_idf_train

<900000x256566 sparse matrix of type '<class 'numpy.float64'>'
	with 64375991 stored elements in Compressed Sparse Row format>

In [30]:
tf_idf_test_cr = tf_idf_vectorizer_cr.transform(pd_user_ad_list_test['creative_id'].values)
tf_idf_test_ad = tf_idf_vectorizer_ad.transform(pd_user_ad_list_test['ad_id'].values)
tf_idf_test_pr = tf_idf_vectorizer_pr.transform(pd_user_ad_list_test['product_id'].values)
tf_idf_test_pc = tf_idf_vectorizer_pc.transform(pd_user_ad_list_test['product_category'].values)
tf_idf_test_av = tf_idf_vectorizer_av.transform(pd_user_ad_list_test['advertiser_id'].values)
tf_idf_test_in = tf_idf_vectorizer_in.transform(pd_user_ad_list_test['industry'].values)

In [31]:
tf_idf_test = sparse.hstack((tf_idf_test_cr,tf_idf_test_pr,tf_idf_test_pc,tf_idf_test_av,tf_idf_test_in)).tocsr()

In [32]:
tf_idf_test

<1000000x256566 sparse matrix of type '<class 'numpy.float64'>'
	with 71544646 stored elements in Compressed Sparse Row format>

## 建模

In [33]:
from sklearn.model_selection import train_test_split
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import accuracy_score

In [39]:
from sklearn.linear_model import LogisticRegression

In [40]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB

In [49]:
from lightgbm.sklearn import LGBMClassifier

### 性别

In [35]:
def model_test(clf,df_data,tf_idf_feature,label_name,test_size=0.3,refit=False):
    '''
    建模测试
    Parameters
    ----------
    clf:sklearn model
    df_data：pandas.DataFrame,训练数据源
    tf_idf_feature：sparse matrix
    label_name：str,标签列名
    test_size：验证集比例
    refit：boolean,测试完是否refit
    Returns
    ----------
    clf:sklearn model
    '''
    X_train, X_test, y_train, y_test = train_test_split(tf_idf_feature,df_data[label_name],test_size=test_size,random_state=0)
    print("数据集划分完毕")
    clf.fit(X_train,y_train)
    print("模型训练完毕")
    y_test_predict = clf.predict(X_test)
    y_train_predict = clf.predict(X_train)
    print("训练集：%f,测试集：%f" % (accuracy_score(y_train,y_train_predict),accuracy_score(y_test,y_test_predict)))
    if refit is True:
        clf.fit(tf_idf_feature,df_data[label_name])
    return clf

#### lr

tf-idf['creative_id']:min_df=30

In [36]:
clf_lr = LogisticRegression(max_iter=1000,random_state=0)

In [70]:
clf_lr = model_test(clf_lr,pd_user_ad_list_train,tf_idf_train,label_name='gender',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.932446,测试集：0.908770


tf-idf['creative_id','product_id','product_category','advertiser_id','industry']：min_df=30

In [37]:
clf_lr = model_test(clf_lr,pd_user_ad_list_train,tf_idf_train,label_name='gender',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.944122,测试集：0.921567


tf-idf:['creative_id','ad_id','product_id','product_category','advertiser_id','industry']:min_df=30，重复点击次数

In [267]:
clf_lr = model_test(clf_lr,pd_user_ad_list_train,tf_idf_train,label_name='gender',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.950727,测试集：0.920048


tf-idf:['creative_id','product_id','product_category','advertiser_id','industry']:min_df=30，重复点击次数

In [272]:
clf_lr = model_test(clf_lr,pd_user_ad_list_train,tf_idf_train,label_name='gender',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.943244,测试集：0.920696


refit

In [299]:
clf_lr_gender = LogisticRegression(max_iter=1000)

In [300]:
clf_lr_gender.fit(tf_idf_train,pd_user_ad_list_train['gender'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

#### 多项式贝叶斯

In [56]:
clf_mnb= MultinomialNB()

tf['creative_id']:min_df=1

In [85]:
clf_mnb = model_test(clf_mnb,pd_user_ad_list_train,tf_idf_train,label_name='gender',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.886489,测试集：0.857737


tf['creative_id']:min_df=10

In [90]:
clf_mnb = model_test(clf_mnb,pd_user_ad_list_train,tf_idf_train,label_name='gender',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.923608,测试集：0.906515


tf['creative_id']:min_df=30

In [108]:
clf_mnb = model_test(clf_mnb,pd_user_ad_list_train,tf_train,label_name='gender',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.917257,测试集：0.908052


tf['creative_id']:min_df=100

In [97]:
clf_mnb = model_test(clf_mnb,pd_user_ad_list_train,tf_idf_train,label_name='gender',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.911303,测试集：0.905263


tf-idf['creative_id']:min_df=30

In [57]:
clf_mnb = model_test(clf_mnb,pd_user_ad_list_train,tf_idf_train,label_name='gender',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.920537,测试集：0.909656


tf-idf['creative_id']:min_df=30,加上点击次数

In [262]:
clf_mnb = model_test(clf_mnb,pd_user_ad_list_train,tf_idf_train_cr,label_name='gender',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.919586,测试集：0.908252


tf-idf['creative_id','product_id','product_category','advertiser_id','industry']:min_df=30

In [230]:
clf_mnb = model_test(clf_mnb,pd_user_ad_list_train,tf_idf_train,label_name='gender',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.906313,测试集：0.900652


tf-idf['creative_id','ad_id','product_id','product_category','advertiser_id','industry']:min_df=30，加上点击次数

In [263]:
clf_mnb = model_test(clf_mnb,pd_user_ad_list_train,tf_idf_train,label_name='gender',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.912256,测试集：0.905644


#### 伯努利贝叶斯

tf-idf['creative_id']：min_df=30

In [41]:
clf_bnb = BernoulliNB()

In [109]:
clf_bnb = model_test(clf_bnb,pd_user_ad_list_train,tf_train,label_name='gender',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.919308,测试集：0.909885


tf-idf['creative_id']：min_df=1

In [84]:
clf_bnb = model_test(clf_bnb,pd_user_ad_list_train,tf_idf_train,label_name='gender',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.926208,测试集：0.895704


tf-idf['creative_id','product_id','product_category','advertiser_id','industry']:min_df=30

In [231]:
clf_bnb = model_test(clf_bnb,pd_user_ad_list_train,tf_idf_train,label_name='gender',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.887865,测试集：0.882570


#### LGBM

In [314]:
lgbm_gender = LGBMClassifier(n_estimators=300,learning_rate=0.1,max_depth=8,num_leaves=127,subsample=0.8,colsample_bytree=0.8,min_split_gain=3,random_state=0,reg_alpha=0.05)

In [ ]:
lgbm_gender.fit()

In [ ]:
lgbm_gender = LGBMClassifier()

In [315]:
lgbm_gender = model_test(lgbm_gender,pd_user_ad_list_train,tf_idf_train,label_name='gender',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.798248,测试集：0.795911


### 年龄

#### 多项式贝叶斯

tf-idf['creative_id']：min_df=30

In [59]:
clf_mnb = model_test(clf_mnb,pd_user_ad_list_train,tf_idf_train,label_name='age',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.506217,测试集：0.357393


#### 伯努利贝叶斯

tf-idf['creative_id']：min_df=30

In [72]:
clf_bnb = model_test(clf_bnb,pd_user_ad_list_train,tf_idf_train,label_name='age',test_size=0.3)

数据集划分完毕
模型训练完毕
训练集：0.547689,测试集：0.368556


#### lr

tf-idf['creative_id']：min_df=30

In [238]:
clf_lr = LogisticRegression(max_iter=1000)

In [76]:
clf_lr = model_test(clf_lr,pd_user_ad_list_train,tf_idf_train,label_name='age',test_size=0.3)

数据集划分完毕


/opt/app/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


模型训练完毕
训练集：0.622743,测试集：0.386215


tf-idf:['creative_id','product_id','product_category','advertiser_id','industry']:min_df=30

In [239]:
clf_lr = model_test(clf_lr,pd_user_ad_list_train,tf_idf_train,label_name='age',test_size=0.3)

数据集划分完毕


/opt/app/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


模型训练完毕
训练集：0.673306,测试集：0.421833


In [296]:
clf_lr_age = LogisticRegression(max_iter=1000)

In [298]:
clf_lr_age.fit(tf_idf_train,pd_user_ad_list_train['age'])

/opt/app/conda/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### 保存模型

In [45]:
from sklearn.externals import joblib

/opt/app/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [394]:
joblib.dump(clf_lr_gender,'./model/lr_gender_tfidf_noclicktimes.pkl')

['./model/lr_gender_tfidf_noclicktimes.pkl']

In [397]:
joblib.dump(clf_lr_age,'./model/lr_age_tfidf_noclicktimes.pkl')

['./model/lr_age_tfidf_noclicktimes.pkl']

载入模型

In [46]:
clf_lr_gender = joblib.load("./model/lr_gender_tfidf_noclicktimes.pkl")

In [47]:
train_result_gender = clf_lr_gender.predict(tf_idf_train)

In [48]:
accuracy_score(pd_user_ad_list_train['gender'],train_result_gender)

0.9426955555555555

### 测试集输出结果

性别

In [392]:
train_result_gender = clf_lr_gender.predict(tf_idf_train)

In [393]:
accuracy_score(pd_user_ad_list_train['gender'],train_result_gender)

0.9426955555555555

In [305]:
test_result_gender = clf_lr_gender.predict(tf_idf_test)

In [306]:
pd_user_ad_list_test['predicted_gender'] = test_result_gender

年龄

In [395]:
train_result_age = clf_lr_age.predict(tf_idf_train)

In [396]:
accuracy_score(pd_user_ad_list_train['age'],train_result_age)

0.6492277777777777

In [307]:
test_result_age = clf_lr_age.predict(tf_idf_test)

In [308]:
pd_user_ad_list_test['predicted_age'] = test_result_age

In [312]:
pd_user_ad_list_test['predicted_age'].value_counts()/pd_user_ad_list_test['predicted_age'].value_counts().sum()

3     0.276018
2     0.179521
4     0.159956
5     0.139169
6     0.110705
7     0.057754
1     0.025161
8     0.023479
9     0.016450
10    0.011787
Name: predicted_age, dtype: float64

In [311]:
pd_user_ad_list_train['age'].value_counts()/pd_user_ad_list_train['age'].value_counts().sum()

3     0.225454
4     0.167309
2     0.165857
5     0.145186
6     0.113022
7     0.074123
1     0.039106
8     0.035519
9     0.021638
10    0.012787
Name: age, dtype: float64

In [313]:
pd_user_ad_list_test[['user_id','predicted_age','predicted_gender']].to_csv("./result/submission.csv",index=False)